In [1]:
import pandas as pd

In [2]:
def get_data(file_name:str) -> pd.DataFrame:
    df = pd.read_excel(file_name)
    df.columns = df.iloc[1]
    df = df[2:]
    return df

In [3]:
df = get_data('data.xlsx')
df

1,KolonSiraNo,TabloAd,KolonAd,VeriTipi,VeriUzunluk,VeriKurus,PK,KolonAciklama
2,1,Gorevlendirme,Gorev_Id,int,4,-,0,NaN
3,2,Gorevlendirme,Drm_Kod,char,1,-,0,NaN
4,3,Gorevlendirme,Tlp_Brm_Kod,smallint,2,-,0,NaN
5,4,Gorevlendirme,Tlp_Sicil_No,char,8,-,0,NaN
6,5,Gorevlendirme,Tlp_Pers_Ad_Soyad,nvarchar,82,-,0,NaN
7,6,Gorevlendirme,Isl_Brm_Kod,smallint,2,-,0,NaN
8,7,Gorevlendirme,Isl_Grv_Kod,char,8,-,0,NaN


In [4]:
def convert_dtype(df:pd.DataFrame, db_type:str) -> pd.DataFrame:
    db2_dtype_mapping = {'char':'Varchar2',
                         'varchar':'Varchar2',
                         'time':'Varchar2(15)',
                         'smallint':'Number(5)',
                         'integer':'Number(10)',
                         'bigint':'Number(19)',
                         'timestmp':'Timestamp(6)',
                         'date':'Date',
                         'boolean':'Number(1)',
                         'decimal':'Number',
                         'numeric':'Number',
                         'double':'Binary_Double',
                         'float':'Binary_Double',
                         'real':'Binary_Double',
                         'int':'Number(10)',
                         'nvarchar':'Varchar2'
                        }

    mssql_dtype_mapping = {'char':'Varchar2',
                           'varchar':'Varchar2',
                           'smallint':'Number(5)',
                           'integer':'Number(10)',
                           'bigint':'Number(20)',
                           'datetime':'Date',
                           'decimal':'Number',
                           'numeric':'Number',
                           'float':'Float(53)',
                           'real':'Float(24)',
                           'bit':'Number(3)',
                           'money':'Number(19,4)',
                           'tinyint':'Number(3)',
                           'text':'Long',
                           'timestamp':'Raw',
                           'uniqueidentifier':'Varchar2(36)',
                           'int':'Number(10)',
                           'nvarchar':'Varchar2'
                          }
    
    if db_type == 'db2':
        df['YeniVeriTipi'] = df['VeriTipi'].map(db2_dtype_mapping).fillna(df['VeriTipi'])

    if db_type == 'mssql':
        df['YeniVeriTipi'] = df['VeriTipi'].map(mssql_dtype_mapping).fillna(df['VeriTipi'])

    return df

In [5]:
df = convert_dtype(df, 'mssql')
df

1,KolonSiraNo,TabloAd,KolonAd,VeriTipi,VeriUzunluk,VeriKurus,PK,KolonAciklama,YeniVeriTipi
2,1,Gorevlendirme,Gorev_Id,int,4,-,0,NaN,Number(10)
3,2,Gorevlendirme,Drm_Kod,char,1,-,0,NaN,Varchar2
4,3,Gorevlendirme,Tlp_Brm_Kod,smallint,2,-,0,NaN,Number(5)
5,4,Gorevlendirme,Tlp_Sicil_No,char,8,-,0,NaN,Varchar2
6,5,Gorevlendirme,Tlp_Pers_Ad_Soyad,nvarchar,82,-,0,NaN,Varchar2
7,6,Gorevlendirme,Isl_Brm_Kod,smallint,2,-,0,NaN,Number(5)
8,7,Gorevlendirme,Isl_Grv_Kod,char,8,-,0,NaN,Varchar2


In [ ]:
schema = 'wods5'
length_contained_types = ['Varchar2(15)', 'Number(5)', 'Number(10)', 'Number(19)', 'Timestamp(6)',
                          'Number(1)', 'Number(20)', 'Float(53)', 'Float(24)', 'Number(3)',
                          'Number(19,4)', 'Varchar2(36)']

sql_script = f"CREATE TABLE wods5.{df.iloc[0]['TabloAd']} (\n"

for index, row in df.iterrows():
    col_name = row['KolonAd']
    data_type = row['YeniVeriTipi']

    if data_type in length_contained_types:
        data_length = data_type

    else:
        data_length = f'{data_type}({row["VeriUzunluk"]})'
        
    sql_script += f"{col_name}\t{data_length},\n"

# Add DWH Date Time
sql_script += "\tVA_AKTAR_TAR DATE DEFAULT trunc(sysdate),\n"
sql_script += "\tVA_AKTAR_ZMN VARCHAR2(15 BYTE) DEFAULT to_CHAR(sysdate, 'HH24:MI:SS')\n"
sql_script += "\t) TABLESPACE TBS_WODS5;"

print(sql_script)


In [9]:
def generate_sql_script(df, schema:str):
    length_contained_types = ['Varchar2(15)', 'Number(5)', 'Number(10)', 'Number(19)', 'Timestamp(6)',
                              'Number(1)', 'Number(20)', 'Float(53)', 'Float(24)', 'Number(3)',
                              'Number(19,4)', 'Varchar2(36)']

    sql_script = f"CREATE TABLE wods5.{df.iloc[0]['TabloAd']} (\n"

    for index, row in df.iterrows():
        col_name = row['KolonAd']
        data_type = row['YeniVeriTipi']

        if data_type in length_contained_types:
            data_length = data_type
        else:
            data_length = f'{data_type}({row["VeriUzunluk"]})'
        
        sql_script += f"\t{col_name}{data_length},\n"

    # Add DWH Date Time
    sql_script += "\tVA_AKTAR_TAR DATE DEFAULT trunc(sysdate),\n"
    sql_script += "\tVA_AKTAR_ZMN VARCHAR2(15 BYTE) DEFAULT to_CHAR(sysdate, 'HH24:MI:SS')\n"
    sql_script += "\t) TABLESPACE TBS_WODS5;"

    return sql_script

In [10]:
script = generate_sql_script(df, schema='wods5')
print(script)

CREATE TABLE wods5.Gorevlendirme (
	Gorev_IdNumber(10),
	Drm_KodVarchar2(1),
	Tlp_Brm_KodNumber(5),
	Tlp_Sicil_NoVarchar2(8),
	Tlp_Pers_Ad_SoyadVarchar2(82),
	Isl_Brm_KodNumber(5),
	Isl_Grv_KodVarchar2(8),
	VA_AKTAR_TAR DATE DEFAULT trunc(sysdate),
	VA_AKTAR_ZMN VARCHAR2(15 BYTE) DEFAULT to_CHAR(sysdate, 'HH24:MI:SS')
	) TABLESPACE TBS_WODS5;


In [13]:
def run():
    df = get_data(file_name='data.xlsx')
    df = convert_dtype(df, db_type='mssql')
    script = generate_sql_script(df, schema='wods5')
    print(script)
run()

CREATE TABLE wods5.Gorevlendirme (
	Gorev_IdNumber(10),
	Drm_KodVarchar2(1),
	Tlp_Brm_KodNumber(5),
	Tlp_Sicil_NoVarchar2(8),
	Tlp_Pers_Ad_SoyadVarchar2(82),
	Isl_Brm_KodNumber(5),
	Isl_Grv_KodVarchar2(8),
	VA_AKTAR_TAR DATE DEFAULT trunc(sysdate),
	VA_AKTAR_ZMN VARCHAR2(15 BYTE) DEFAULT to_CHAR(sysdate, 'HH24:MI:SS')
	) TABLESPACE TBS_WODS5;
